In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import mixture
sns.set(style='whitegrid')
sns.set_palette('bright')
import warnings
warnings.filterwarnings('ignore')
import collections
from scipy.spatial import distance


In [2]:
df = pd.read_csv('preci_alt.csv')
df.columns = ['date','value']
df.date = pd.to_datetime(df.date,format='%b %d, %Y')
df.head(3)

,date,value
0,2010-01-01,0.0
1,2010-01-02,0.0
2,2010-01-03,0.0


In [3]:
df19 = df[df.date.dt.year==2019]

In [4]:
def gmm2k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    mins = min(best_gmmq1.n_components,best_gmmq2.n_components)
    if best_gmmq1.n_components > best_gmmq2.n_components:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    else:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    #print(best_gmmq1.get_params())
    #print(best_gmmq2.get_params())
    #print(best_gmmq1.means_)
    #print(best_gmmq2.means_)
    #print(best_gmmq1.weights_)
    #print(best_gmmq2.weights_)
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(best_gmmq1.weights_,means))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(best_gmmq2.weights_,means))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    #print(od1)
    #print(od2)
    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(od1[tt][1] - od2[tt][1])
        diffweights.append(od1[tt][0] - od2[tt][0]) 
    print(diffweights)
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i] * 10000)
    return od1,od2,sum(r)

In [5]:
def gmm10k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    
    #print(best_gmmq1.get_params())
    #print(best_gmmq2.get_params())
    #print(best_gmmq1.means_)
    #print(best_gmmq2.means_)
    #print(best_gmmq1.weights_)
    #print(best_gmmq2.weights_)
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(best_gmmq1.weights_,means))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(best_gmmq2.weights_,means))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    maxx = max(len(od1),len(od2))
    diff = abs(len(od1) - len(od2))
    if len(od2) > len(od1):
        for i in range(0,diff):
            od1.append((0,0))
    elif len(od2) < len(od1):
        for i in range(0,diff):
            od2.append((0,0))
    #print(maxx)
    #print(od1)
    #print(od2)
    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(od1[tt][1] - od2[tt][1])
        diffweights.append(od1[tt][0] - od2[tt][0]) 
    #print(diffweights)
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i])
    print(len(od1) , len(od2))
    return od1,od2,sum(r)

In [6]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm10k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

3 3
[(0.932444691104159, 0.009094296424982948), (0.06755530889584141, 0.03682632135568748), (0, 0)]
[(0.6427045173822166, 0.0017537042888375557), (0.29050665528132086, 0.00850540962263617), (0.06678882733646238, 0.05143587184378963)]
-0.0007519793975241493


In [7]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm2k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

[1.9389657670121352e-05, -1.9389657669580118e-05]
[(0.932444691104159, 0.009094296424982948), (0.06755530889584141, 0.03682632135568748)]
[(0.9324253014464888, 0.003844462857303236), (0.06757469855351099, 0.05103476550445302)]
0.0037728934376433385


In [8]:
a = np.zeros(shape=(12, 12))
for i in range(1,13):
    for j in range(1,13):
        dfc = df19[df19.date.dt.month == i]
        dfd = df19[df19.date.dt.month == j]
        _,_,r = gmm10k(dfc.value,dfd.value)
        a[i-1][j-1] = r
a

1 1
1 1
1 1
1 1
1 1
3 3
2 2
3 3
2 2
3 3
2 2
2 2
1 1
1 1
1 1
1 1
1 1
3 3
2 2
3 3
2 2
3 3
2 2
2 2
1 1
1 1
1 1
1 1
1 1
3 3
2 2
3 3
2 2
3 3
2 2
2 2
1 1
1 1
1 1
1 1
1 1
3 3
2 2
3 3
2 2
2 2
2 2
2 2
1 1
1 1
1 1
1 1
1 1
3 3
2 2
3 3
2 2
3 3
2 2
2 2
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
2 2
2 2
2 2
2 2
2 2
3 3
2 2
3 3
2 2
3 3
2 2
2 2
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
2 2
2 2
2 2
2 2
2 2
3 3
2 2
3 3
2 2
3 3
2 2
2 2
3 3
3 3
3 3
2 2
2 2
3 3
2 2
3 3
2 2
3 3
2 2
3 3
2 2
2 2
2 2
2 2
2 2
3 3
2 2
3 3
2 2
3 3
2 2
2 2
2 2
2 2
2 2
2 2
2 2
3 3
2 2
3 3
2 2
2 2
2 2
2 2


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.27962904e-03,
         1.86389899e-03,  1.23351598e-02,  7.42092362e-03,
         3.45175380e-03,  4.90084299e-04,  3.26828713e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.27962904e-03,
         1.87469604e-03,  1.23351598e-02,  7.42092362e-03,
         3.45175380e-03,  4.90084299e-04,  3.26828713e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.26154842e-03,
         1.86389899e-03,  1.23351598e-02,  7.42092362e-03,
         3.45175380e-03,  4.90084299e-04,  3.26828713e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.41063739e-03,
         1.89949822e-03,  1.24940828e-02,  7.56996656e-03,
         3.75511058e-03,  5.26619396e-04,  3.60593568e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.0000000

In [10]:
dfw = pd.DataFrame(a)
cm = sns.light_palette("green", as_cmap=True)
s = dfw.style.background_gradient(cmap=cm)
s

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005280,0.001864,0.012335,0.007421,0.003452,0.000490,0.000327
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.005280,0.001875,0.012335,0.007421,0.003452,0.000490,0.000327
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.005262,0.001864,0.012335,0.007421,0.003452,0.000490,0.000327
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.005411,0.001899,0.012494,0.007570,0.003755,0.000527,0.000361
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.005280,0.001875,0.012335,0.007421,0.003452,0.000490,0.000327
5,0.005280,0.005262,0.005280,0.005411,0.005280,0.000000,0.000786,0.000751,0.004896,0.000618,0.003673,0.003964
6,0.001873,0.001864,0.001864,0.001898,0.001864,0.001534,-0.000000,0.003380,-0.003211,-0.001896,-0.000732,-0.000593
7,0.012335,0.012335,0.012335,0.012494,0.012335,0.000751,0.003380,0.000000,0.006130,0.002930,0.009269,0.009748
8,0.007413,0.007421,0.007421,0.007570,0.007413,0.003876,-0.003263,0.006131,-0.000000,0.002230,0.004093,0.004624
9,0.003452,0.003452,0.003447,0.003755,0.003608,0.000159,-0.004962,0.006176,0.000421,0.000939,0.001359,0.001943


## OrderbyMeans

In [8]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(means,best_gmmq1.weights_))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1

OrderedDict([(300.04340106175016, 0.6666714159661873),
             (34.362525668641766, 0.3333285840338131)])

In [9]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
x = dict(zip(means,best_gmmq2.weights_))
od2 = collections.OrderedDict(sorted(x.items(),reverse=True))
od2

OrderedDict([(272.25874282362764, 0.32922962293822033),
             (75.06318767078143, 0.6707703770617822)])

## OrderbyWeights

In [10]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(best_gmmq1.weights_,means))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1 = list(zip(list(od1.keys()),list(od1.values())))
od1

[(0.6666714159661873, 300.04340106175016),
 (0.3333285840338131, 34.362525668641766)]

In [11]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
y = dict(zip(best_gmmq2.weights_,means))
od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
od2 = list(zip(list(od2.keys()),list(od2.values())))
od2

[(0.6707703770617822, 75.06318767078143),
 (0.32922962293822033, 272.25874282362764)]

In [12]:
thres = 0.05
diffmean = []
diffweights = []
for i in range(len(od1)):
    diffmean.append(od1[i][1] - od2[i][1])
    diffweights.append(od1[i][0] - od2[i][0])        

In [13]:
print(diffmean)
print(diffweights)

[224.98021339096874, -237.8962171549859]
[-0.004098961095594933, 0.004098961095592768]
